In [1]:
import os
import pandas as pd
import numpy as np

from datetime import datetime
import json
import ast

from numpy import reshape

import matplotlib.pyplot as plt

from sklearn.cluster import KMeans, DBSCAN
from sklearn import metrics
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

from scipy.stats import poisson, expon, nbinom

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

In [2]:
path_in = 'D:/flc/shop/raw_data/extracted_data'
path_out = 'D:/flc/shop/mapped_data'

In [3]:
df_shop = pd.read_csv(r'D:\CHIPHISHOP\VD_CID_SHOP_LC.csv')

In [4]:
df_ward_population = pd.read_csv(r'D:\CHIPHISHOP\VD_CID_WARD_POPULATION.csv')
df_ward_population['PROVINCE_NAME'].replace({'Cao B?ng': 'Cao Bằng'}, inplace = True)
df_ward_population['WARD_CODE_map'] = df_ward_population['WARD_CODE'].apply(lambda x: str(100000 + x)[1:])

In [5]:
# Fix bug: REGION_CODE_7 conflict with SHOP_ADDRESS
# df_shop.loc[df_shop['SHOP_KEY'] == 186, 'REGION_CODE_7'] = '00008'
df_shop.loc[df_shop['SHOP_KEY'] == 55662, 'REGION_CODE_7'] = '26326'
df_shop.loc[df_shop['SHOP_KEY'] == 418, 'REGION_CODE_7'] = '26020'
df_shop.loc[df_shop['SHOP_KEY'] == 62011, 'REGION_CODE_7'] = '26830'
# df_shop.loc[df_shop['SHOP_KEY'] == 3, 'REGION_CODE_7'] = '09169'
df_shop.loc[df_shop['SHOP_KEY'] == 55654, 'REGION_CODE_7'] = '27106'
# df_shop.loc[df_shop['SHOP_KEY'] == 2700, 'REGION_CODE_7'] = '20198'
# df_shop.loc[df_shop['SHOP_KEY'] == 4132, 'REGION_CODE_7'] = '20197'
df_shop.loc[df_shop['SHOP_KEY'] == 59296, 'REGION_CODE_7'] = '26830'

In [6]:
df_shop = pd.merge(df_shop, df_ward_population, how = 'left', left_on = 'REGION_CODE_7', right_on = 'WARD_CODE_map')
df_shop.rename(columns = {'AREA': 'ward_AREA'}, inplace = True)

In [7]:
df_shop_cost = pd.read_excel(r'D:\CHIPHISHOP\CPThueNha_history_v1.xlsx') # How many milions of Vietnam dong/month

In [8]:
df_shop_cost.columns = ['YEAR', 'MONTH', 'SHOP_CODE', 'RENTAL_COST']
df_shop_cost['year_month'] = pd.to_datetime(df_shop_cost['YEAR'].apply(lambda x: str(x)) + '-' + df_shop_cost['MONTH'].apply(lambda x: str(x)))
df_shop_cost['SHOP_CODE'] = df_shop_cost['SHOP_CODE'].apply(lambda x: str(x))

In [9]:
latest_month = '2023-12-01'
df_transaction = pd.read_csv(r'D:\CHIPHISHOP\VF_CID_SALES_TRANSACTION_FLC_Sales.csv')
df_transaction['year_month'] = pd.to_datetime(pd.to_datetime(df_transaction['MIN_TRANSACTION_DATE']).apply(lambda x: str(x.year) + '-' + str(x.month)))
df_transaction = df_transaction[df_transaction['year_month'] <= latest_month]

In [20]:
df_transaction

,SHOP_KEY,DATE_KEY,MIN_TRANSACTION_DATE,MAX_TRANSACTION_DATE,N_BILLS,N_CUSTOMERS,TOTAL_AMOUNT_BY_SHOP_DATE,TOTAL_QTY_BY_SHOP_DATE,N_EMPLOYEES,year_month
0,2,20190721,2019-07-21 07:36:34.997,2019-07-21 21:41:30.830,63,45,"6051120,16450216",524,7,2019-07-01
1,2,20190722,2019-07-22 08:26:52.803,2019-07-22 21:09:36.380,58,45,"5044280,52910053",345,6,2019-07-01
2,2,20190723,2019-07-23 07:38:11.137,2019-07-23 21:45:41.913,66,49,"5279694,87205387",577,6,2019-07-01
3,2,20190724,2019-07-24 07:19:41.017,2019-07-24 21:26:36.750,73,57,"12032735,5363155",437,8,2019-07-01
4,2,20190725,2019-07-25 07:19:56.967,2019-07-25 21:06:35.867,74,58,"9948313,6979317",674,8,2019-07-01
...,...,...,...,...,...,...,...,...,...,...
493995,67507,20230329,2023-03-29 07:14:44.370,2023-03-29 21:38:51.823,92,84,"17335845,8008658",578,4,2023-03-01
493996,67507,20230330,2023-03-30 07:59:18.813,2023-03-30 21:46:19.693,109,91,"15184341,9480519",857,3,2023-03-01
493997,67507,20230331,2023-03-31 07:08:11.740,2023-03-31 21:56:59.660,89,73,"8843696,40692641",764,3,2023-03-01
493998,67617,20230331,2023-03-31 11:25:01.103,2023-03-31 21:27:50.157,79,62,"8549934,67532467",536,2,2023-03-01


In [10]:
df_transaction_shop = pd.merge(df_transaction, df_shop, how = 'left', on = 'SHOP_KEY')
df_transaction_shop['first_transaction_date'] = df_transaction_shop.groupby(['SHOP_KEY'])['MIN_TRANSACTION_DATE'].transform('min')
df_transaction_shop = df_transaction_shop[df_transaction_shop['OPEN_DATE'] < latest_month + ' 00:00:00.000']
df_transaction_shop = df_transaction_shop[df_transaction_shop['OPEN_DATE'] >= min(df_transaction_shop['MIN_TRANSACTION_DATE'])]

In [11]:
df_transaction_shop = pd.merge(df_transaction_shop, df_shop_cost, how = 'left', on = ['SHOP_CODE', 'year_month'])
df_transaction_shop['open_month'] = pd.to_datetime(pd.to_datetime(df_transaction_shop['OPEN_DATE']).apply(lambda x: str(x.year) + '-' + str(x.month)))
missing_rental_cost_shop_keys = df_transaction_shop[(df_transaction_shop['year_month'] >= df_transaction_shop['open_month']) & (df_transaction_shop['RENTAL_COST'].isna())]['SHOP_KEY'].unique()
df_transaction_shop = df_transaction_shop[df_transaction_shop['SHOP_KEY'].isin(missing_rental_cost_shop_keys) == False]
df_transaction_shop = df_transaction_shop[df_transaction_shop['MIN_TRANSACTION_DATE'] >= df_transaction_shop['OPEN_DATE']]

In [12]:
p_rank = 0.5
df_tmp = df_transaction_shop[['SHOP_KEY', 'year_month', 'open_month', 'RENTAL_COST']].sort_values(by = ['SHOP_KEY', 'year_month']).drop_duplicates()
df_tmp['rental_cost_lead1'] = df_tmp.groupby(['SHOP_KEY'])['RENTAL_COST'].shift(-1)
df_tmp['rental_cost_lag1'] = df_tmp.groupby(['SHOP_KEY'])['RENTAL_COST'].shift(1)
df_tmp['rental_cost_diff_p'] = (df_tmp['RENTAL_COST'] - df_tmp['rental_cost_lead1'])/df_tmp['rental_cost_lead1']
df_tmp['rental_cost_imputed'] = df_tmp['RENTAL_COST']
df_tmp.loc[(df_tmp['RENTAL_COST'] < 0) & (df_tmp['rental_cost_lead1'] >= 0), 'rental_cost_imputed'] = df_tmp['rental_cost_lead1']
df_tmp.loc[(df_tmp['RENTAL_COST'] < 0) & (df_tmp['rental_cost_lead1'] < 0) & (df_tmp['rental_cost_lag1'] >= 0), 'rental_cost_imputed'] = df_tmp['rental_cost_lag1']
df_tmp.loc[(df_tmp['RENTAL_COST'] < 0) & (df_tmp['rental_cost_lead1'].isna()) & (df_tmp['rental_cost_lag1'] >= 0), 'rental_cost_imputed'] = df_tmp['rental_cost_lag1']
df_tmp.loc[(df_tmp['year_month'] == df_tmp['open_month']) & (df_tmp['rental_cost_diff_p'] > p_rank) & (df_tmp['rental_cost_lead1'] >= 0), 'rental_cost_imputed'] = df_tmp['rental_cost_lead1']
df_transaction_shop = pd.merge(df_transaction_shop, df_tmp[['SHOP_KEY', 'year_month', 'rental_cost_imputed']], how = 'left', on = ['SHOP_KEY', 'year_month'])

In [13]:
df_out = df_transaction_shop[['SHOP_KEY', 'year_month', 'open_month', 'RENTAL_COST', 'rental_cost_imputed']].drop_duplicates()
df_out.columns = ['SHOP_KEY', 'transaction_year_month', 'shop_open_month', 'rental_cost', 'rental_cost_imputed']

In [14]:
df_out['rental_cost'].min()

-43.333333

In [15]:
df_out['rental_cost_imputed'].min()

0.0

In [17]:
df_out['rental_cost_imputed'].value_counts().sum() == df_out['rental_cost_imputed'].shape[0]

True

In [18]:
df_out.to_csv(r'D:\CHIPHISHOP\df_rental_cost_imputed.csv',index = False)

In [19]:
df_out

,SHOP_KEY,transaction_year_month,shop_open_month,rental_cost,rental_cost_imputed
0,2,2019-07-01,2019-07-01,49.500000,49.500000
11,2,2019-08-01,2019-07-01,49.500000,49.500000
42,2,2019-09-01,2019-07-01,49.500000,49.500000
72,2,2019-10-01,2019-07-01,49.500000,49.500000
103,2,2019-11-01,2019-07-01,49.500000,49.500000
...,...,...,...,...,...
419048,64703,2023-03-01,2023-02-01,27.777778,27.777778
419079,64737,2023-02-01,2023-02-01,28.888889,28.888889
419087,64737,2023-03-01,2023-02-01,28.888889,28.888889
419118,64749,2023-02-01,2023-02-01,30.000000,30.000000
